# MLMAPPER USING NEURAL NETWORK

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

### DATA PATH

In [40]:
#paths to data 
albedo="/home/gokul/g0kul6/ml4sci/Data_Albedo/Albedo_Map.csv"
LPFe_Map="/home/gokul/g0kul6/ml4sci/Data_Albedo/LPFe_Map.csv"
LPK_Map="/home/gokul/g0kul6/ml4sci/Data_Albedo/LPK_Map.csv"
LPTh_Map="/home/gokul/g0kul6/ml4sci/Data_Albedo/LPTh_Map.csv"
LPTi_Map="/home/gokul/g0kul6/ml4sci/Data_Albedo/LPTi_Map.csv"

### CUSTOM DATASET TO GET INPUT,GT

In [41]:
#custom dataset
class dataset(Dataset):
  def __init__(self,path_1,path_2,path_3,path_4,path_5):
    self.path_1=path_1
    self.path_2=path_2
    self.path_3=path_3
    self.path_4=path_4
    self.path_5=path_5
    self.X_1=torch.FloatTensor(np.array(pd.read_csv(path_1)))
    self.X_2=torch.FloatTensor(np.array(pd.read_csv(path_2)))
    self.X_3=torch.FloatTensor(np.array(pd.read_csv(path_3)))
    self.X_4=torch.FloatTensor(np.array(pd.read_csv(path_4)))
    self.X_5=torch.FloatTensor(np.array(pd.read_csv(path_5)))
    l=int(np.shape(self.X_5)[1]/2)
    self.X_1=(self.X_1[0:l].flatten()-torch.mean(self.X_1[0:l].flatten()))/torch.std(self.X_1[0:l].flatten())
    self.X_2=(self.X_2[0:l].flatten()-torch.mean(self.X_2[0:l].flatten()))/torch.std(self.X_2[0:l].flatten())
    self.X_3=(self.X_3[0:l].flatten()-torch.mean(self.X_3[0:l].flatten()))/torch.std(self.X_3[0:l].flatten())
    self.X_4=(self.X_4[0:l].flatten()-torch.mean(self.X_4[0:l].flatten()))/torch.std(self.X_4[0:l].flatten())
    self.X_5=self.X_5[0:l].flatten()
    self.X=torch.stack((self.X_1,self.X_2,self.X_3,self.X_4),1)  
    self.Y=self.X_5
  
  def __len__(self):
    self.filelength=len(self.Y)
    return self.filelength

  def __getitem__(self,idx):
    return self.X[idx],self.Y[idx]

### NEURAL NETWORK MODEL

In [42]:
#neural network model
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden)
        self.hidden2 = torch.nn.Linear(n_hidden,n_hidden)
        self.hidden3 = torch.nn.Linear(n_hidden,n_hidden)    
        self.predict = torch.nn.Linear(n_hidden, n_output)   
        self.dropout = torch.nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.dropout(F.relu(self.hidden1(x)))
        x = self.dropout(F.relu(self.hidden2(x)))
        x = self.dropout(F.relu(self.hidden3(x)))      
        x = self.predict(x)            
        return x



### TRAIN 


In [45]:
class train():
    def __init__(self,batch_size,epochs,lr,train_val_split):
        self.batch_size=batch_size
        self.epochs=epochs
        self.lr=lr
        self.train_val_split=train_val_split
        self.data=dataset(LPFe_Map,LPK_Map,LPTh_Map,LPTi_Map,albedo)
        self.train_data,self.val_data=random_split(self.data,[len(self.data)-int(self.train_val_split*len(self.data)),int(self.train_val_split*len(self.data))],generator=torch.Generator().manual_seed(42))
        self.train_loader=DataLoader(self.train_data,batch_size=self.batch_size,shuffle=True)
        self.val_loader=DataLoader(self.val_data,batch_size=self.batch_size,shuffle=True)
        self.net=Net(n_feature=4, n_hidden=10, n_output=1)  
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=self.lr)
        self.loss_func = torch.nn.MSELoss() 
        self.writer = SummaryWriter()
    def trainer(self):
        self.net=self.net.train()
        self.net=self.net.cuda()
        for epoch in range(self.epochs):
            for input,gt in self.train_loader:
                input = input.cuda()
                gt = gt.cuda()
                gt=torch.reshape(gt,(len(gt),1))
                output = self.net(input)
                loss = self.loss_func(output, gt)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            print('Epoch : {},  train loss : {}'.format(epoch+1,loss.item()))
            with torch.no_grad():
                for input,gt in self.val_loader:
                    input=input.cuda()
                    gt= gt.cuda()
                    gt=torch.reshape(gt,(len(gt),1))
                    val_output = self.net(input)
                    val_loss = self.loss_func(val_output,gt)
            print('Epoch : {},  val_loss : {}'.format(epoch+1,val_loss.item()))
        self.writer.add_scalar("Loss/train", loss, epoch)
        self.writer.add_scalar("Loss/val", val_loss, epoch)
        torch.save(self.net.state_dict(),f"albedo_{self.epochs}_{self.lr}.pth")